# cadCAD Experiment

In [1]:
from model.run import run

df = run()

print(df)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (28, 6, 1, 7)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.05s
     allocated_funds  unallocated_funds committed_brokers  \
5                  0         158.237269                {}   
10                10         166.294803                {}   
15                20         168.323090                {}   
20                30         176.144705                {}   
25                40         167.456606                {}   
30                50         159.834311                {}   
35                60         150.372854                {}   
40                7